# Alternating Layered Shadow Optimization

This notebook presents the code to implement the Alternating Layered Shadow Optimization (ALSO) algorithm. Using ALSO, we can implement many Variational Quantum Algorithms (VQA) involving the Alternanting Layered Ansatz (ALA) classically efficiently using the classical shadows of the input quantum states. For any $n$-qubit state $\rho$, ALA ansatz $U(\boldsymbol{\theta})$ and observable $O=\sum\limits_{i=1}^M O_i$, where $O_i$ are $k$-local observables, our aim is to optimize the function 

\begin{align}
    f_{\rho,O}(\boldsymbol{\theta}) = \text{tr}(O U(\boldsymbol{\theta}) \rho U(\boldsymbol{\theta}) ^ {\dagger}).
\end{align}

We optimize the function using iterative optimization algorithms. Each function evaluation is estimated as 
\begin{align}
 \hat{f}_{\rho, O}(\boldsymbol{\theta}) = \sum \limits_{i=1}^M \text{tr}( U(\boldsymbol{\theta}) ^ {\dagger} O_i U(\boldsymbol{\theta}) \hat{\rho}_T) = \sum \limits_{i=1}^M \text{tr}( W_{O_i}(\boldsymbol{\theta}) \hat{\rho}_T),
\end{align}

where $\hat{\rho}_T $ is the shadow state, and all $ W_{O_i}(\boldsymbol{\theta})$ are local observables.

The notebook contains code for implementing ALSO and VQA for State Preparation Problem (SPP) as well as Quantum Autoencoder (QA) for $8$ and $30$ qubit instances. Although the majority of the examples use SPSA, in the end, we also provide code for optimization using Powell's method. A few things to keep in mind are:
- We highly recommend to keep the paper as a reference while going through the code since many notations are directly used from the paper.
- The SPSA algorithm used here has the update rule  \begin{align} \boldsymbol{\theta}^{(k+1)} = \boldsymbol{\theta}^{(k)} + a ^ {(k)} g ^ {(k)}.\end{align}. Here, $a ^ {(k)} = k ^ {-a_\alpha}$ and the gradient approximation $g ^ {(k)}$ has the form \begin{align}
    g ^ {(k)} = \frac{f_{\rho, O}(\boldsymbol{\theta} + c^{(k)} \Delta ^ {(k)}) - f_{\rho, O}(\boldsymbol{\theta} - c^{(k)} \Delta ^ {(k)})}{2 c ^ {(k)}} \Delta ^ {(k)}, \end{align}  where $c ^ {(k)} = k ^ {-c_\alpha}$ and $ \Delta ^ {(k)}$ is a vector with each entry selected from $\{-1,1 \}$ uniformly.
- n_qubits (number of qubits) and n_B variables should be even numbers due to the nature of how they are used inside the functions and defined in ALA.
- u_depth (depth of $U(\boldsymbol{\theta})$) should be less than n_qubits$/2$. In practice, we require u_depth to be $\mathcal{O}(\log n)$, so this condition is usually always satisfied.
- We recommend changing the number of shadows being loaded from 500000 to 100000 to see faster, but slightly worse, results.


# Install Qiskit

Qiskit is required to compute infidelity between $U(\boldsymbol{\theta}) ^ {\dagger} |0\rangle$ and computational states using Matrix Product State simulator for circuits involving many qubits.

In [ ]:
!pip install qiskit

# Importing Packages

In [1]:
import numpy as np
from scipy.optimize import minimize
import random
import matplotlib.pyplot as plt
from qiskit import * 
from qiskit.providers.aer import AerSimulator
import copy
from qiskit.providers.aer import QasmSimulator
plt.style.use('ggplot')

# Required Functions and Matrix initializations

In this section, we define the required gates and operators.

In [2]:
H_gate = (0.5 ** (0.5)) * np.matrix([[1,1], 
               [1,-1]])

Y_basis = (1/(2 ** 0.5)) * np.matrix([[1 + 0j, -1j], 
                                      [1 + 0j, 1j]]) # This is the gate HS ^ \dagger

Y_basis_dag = np.conj(Y_basis).T

Id = np.matrix([[1 + 0j, 0 + 0j], 
                [0 + 0j, 1 + 0j]])

ket_0 = np.array([1, 0])

ket_0_density = np.outer(ket_0, ket_0)

clifford_gate_dict = {'H': H_gate, 'Y_basis': Y_basis, 'Y_basis_dag': Y_basis_dag, 'I': Id}

# This function returns a numpy matrix of the form R_Z(theta) for any real number theta
def R_Z(theta, dtype = 'complex128'):
  return np.matrix([[np.exp(-1j * theta/2), 0],[0, np.exp(1j * theta/2)]], dtype=dtype)

# This function returns a numpy matrix of the form R_Y(theta) for any real number theta
def R_Y(theta, dtype = 'complex128'):
  return np.matrix([[np.cos(theta/2), (-1 * np.sin(theta/2))], [np.sin(theta/2), np.cos(theta/2)]], dtype=dtype)

# This function returns a numpy matrix of the form R_X(theta) for any real number theta
def R_X(theta, dtype = 'complex128'):
  return np.matrix([[np.cos(theta/2), ((-1j) * np.sin(theta/2))], [((-1j) * np.sin(theta/2)), np.cos(theta/2)]], dtype=dtype)

# This function returns the matrix form of a CNOT gate
# 1. is_rev: By default, the returned matrix assumes that that the first qubit is the control qubit. Set this to 1 to make second qubit control.
# 2. dtype: Data type of the entries of the returned matrix
def CNOT(is_rev=0, dtype='compelx128'):
  if is_rev == 0:
    return np.matrix([[1 + 0j, 0 + 0j, 0 + 0j, 0 + 0j], 
                  [0 + 0j, 1 + 0j, 0 + 0j, 0 + 0j],
                  [0 + 0j, 0 + 0j, 0 + 0j, 1 + 0j],
                  [0 + 0j, 0 + 0j, 1 + 0j, 0 + 0j]], dtype=dtype)
    
  else:
    return np.matrix([[1 + 0j, 0 + 0j, 0 + 0j, 0 + 0j], 
                      [0 + 0j, 0 + 0j, 0 + 0j, 1 + 0j],
                      [0 + 0j, 0 + 0j, 1 + 0j, 0 + 0j],
                      [0 + 0j, 1 + 0j, 0 + 0j, 0 + 0j]], dtype=dtype)


# This function returns a numipy matrix of a sub circuit with 12 parameters involved.
# 1, theta: A 12 dimensional numpy array
# 2. dtype: Data type of the entries of the returned matrix
def s_gate(theta, dtype='complex128'):
  mat = np.eye(4,dtype=dtype)

  mat = mat @ CNOT(is_rev=1,dtype=dtype)

  mat = mat @ np.kron(R_X(theta[8], dtype=dtype), R_X(theta[11], dtype=dtype)) 
  mat = mat @ np.kron(R_Y(theta[7], dtype=dtype), R_Y(theta[10], dtype=dtype))
  mat = mat @ np.kron(R_X(theta[6], dtype=dtype), R_X(theta[9], dtype=dtype))

  mat = mat @ CNOT(dtype=dtype)

  mat = mat @ np.kron(R_X(theta[2], dtype=dtype), R_X(theta[5], dtype=dtype)) 
  mat = mat @ np.kron(R_Y(theta[1], dtype=dtype), R_Y(theta[4], dtype=dtype))
  mat = mat @ np.kron(R_X(theta[0], dtype=dtype), R_X(theta[3], dtype=dtype))

  return mat

# This function returns the inverse of s_gate.
# 1, theta: A 12 dimensional numpy array
# 2. dtype: Data type of the entries of the returned matrix
def s_gate_inverse(theta, dtype='complex128'):
  mat = np.eye(4, dtype=dtype)
  mat = mat @ np.kron(R_X(-theta[0], dtype=dtype), R_X(-theta[3], dtype=dtype))
  mat = mat @ np.kron(R_Y(-theta[1], dtype=dtype), R_Y(-theta[4], dtype=dtype))
  mat = mat @ np.kron(R_X(-theta[2], dtype=dtype), R_X(-theta[5], dtype=dtype))
  mat = mat @ CNOT(dtype=dtype)

  mat = mat @ np.kron(R_X(-theta[6], dtype=dtype), R_X(-theta[9], dtype=dtype))
  mat = mat @ np.kron(R_Y(-theta[7], dtype=dtype), R_Y(-theta[10], dtype=dtype))
  mat = mat @ np.kron(R_X(-theta[8], dtype=dtype), R_X(-theta[11], dtype=dtype))
  mat = mat @ CNOT(is_rev=1, dtype=dtype)

  return mat

## Tensor operations

Here, we define some required tensor operations

In [3]:
# This function applies a gate on specific qubits of the pure state (any vector) and returns the resulting state using matricisation of the tensor. 
# 1. state: Numpy array (vector)
# 2. qubits_to_act: List of integers (0 is for the first qubit)
# 3. gate: Numpy matrix

def across_qubits_gate(state, qubits_to_act, gate):
  NQubits = int(np.log2(len(state)))

  rest_of_qubits = []
  for i in range(NQubits):
    if i not in qubits_to_act:
      rest_of_qubits.append(i)

  state_tensor = np.reshape(state, tuple([2] * NQubits))
  matricised_tensor = np.reshape(np.ndarray.transpose(state_tensor, qubits_to_act + rest_of_qubits), (2 ** len(qubits_to_act), 2 ** len(rest_of_qubits)))
  answer = np.matmul(gate, matricised_tensor)
  answer = np.asarray(answer)
  p = qubits_to_act + rest_of_qubits

  s = np.zeros(len(p), dtype = int)
  
  for i in list(range(len(p))):
      s[p[i]] = i

  Z = np.ndarray.transpose(np.reshape(answer, tuple([2] * NQubits)), s)
  Z = np.reshape(Z, int(2 ** NQubits))

  return(Z)

# This function applies a gate on specific qubits of a density matrix (any matrix) and returns the resulting density matrix using matricisation of the tensor. 
# 1. state: Numpy matrix
# 2. qubits_to_act: List of integers (0 is for the first qubit)
# 3. gate: Numpy matrix

def rotate_state(state, qubits_to_act, gate):
  NQubits = int(np.log2(state.shape[0]))
  state_vec = state.flatten()
  state_vec = across_qubits_gate(state_vec, qubits_to_act, gate)
  state_vec = across_qubits_gate(state_vec, [NQubits + q for q in qubits_to_act], np.conj(gate))
  state_vec = np.reshape(state_vec, (len(state[0]), len(state[0])))
  return state_vec

## Ansatz and shadow related

In this section, we define the main functions required for the simulations. Here, also_feval is the function that uses ALSO to estimate function evaluations. 

In [4]:



# This function is used to convert an integer to a list of binary values.
# 1. x: Integer value
# 2. BitNo: How many binary bits for x's expansion is required
def binary(x, BitNo):
  format(x, 'b').zfill(BitNo)
  Binlist = [int(y) for y in list(format(x, 'b').zfill(BitNo))]
  return Binlist


# This function applies an ALA on an ensemble and returns the resulting ensemble
# 1. ensemble: A list of 2-tuples of the form [(p_1, \ket{\psi_1}), (p_2, \ket{\psi_2}), ... ,(p_t, \ket{\psi_t})]. Here, p_i are probabilities and \ket{\psi_i} are pure states in the mixture.
# 2. theta_tensor: An order 3 tensor of real parameters
# 3. u_depth: Depth of the ALA
# 4. n_qubits: Number of qubits involved
def u(ensemble, theta_tensor, u_depth, n_qubits):
  new_ensemble = []
  for pair in ensemble:
    theta_position = 0
    state = pair[1]
    for d in range(u_depth):
      for i in range(0, n_qubits, 2):
        state = across_qubits_gate(state, [i + (d%2), (i+1 + (d%2))%n_qubits], s_gate(theta_tensor[int(i/2)][d]))
    new_ensemble.append((pair[0], state))
  return new_ensemble

# This function applies the inverse of an ALA on an ensemble and returns the resulting ensemble
# 1. ensemble: A list of 2-tuples of the form [(p_1, \ket{\psi_1}), (p_2, \ket{\psi_2}), ... ,(p_t, \ket{\psi_t})]. Here, p_i are probabilities and \ket{\psi_i} are pure states in the mixture.
# 2. theta_tensor: An order 3 tensor of real parameters
# 3. u_depth: Depth of the ALA
# 4. n_qubits: Number of qubits involved
def u_inverse(ensemble, theta, u_depth, n_qubits):
  new_ensemble = []
  for pair in ensemble:
    theta_position = 0
    state = pair[1]
    for d in range(u_depth-1, -1, -1):
      for i in range(n_qubits-2, -1, -2):
        state = across_qubits_gate(state, [i + (d%2), (i+1 + (d%2))%n_qubits], s_gate_inverse(theta[int(i/2)][d]))
    new_ensemble.append((pair[0], state))
  return new_ensemble

# This function applies the inverse of an ALA on an ensemble and returns f_{\rho,J}, J = \sum \limits_{i=1}^n_B \ket{0}_i\bra{0}
# 1. ensemble: A list of 2-tuples of the form [(p_1, \ket{\psi_1}), (p_2, \ket{\psi_2}), ... ,(p_t, \ket{\psi_t})]. Here, p_i are probabilities and \ket{\psi_i} are pure states in the mixture.
# 2. theta_tensor: An order 3 tensor of real parameters
# 3. u_depth: Depth of the ALA
# 4. n_qubits: Number of qubits involved
# 5. no_of_copies_per_eval: Number of measurement shots/number of copies per function evaluation. 0 means infinite shots/copies
# 6. n_B: Defined in 1.
def u_inverse_exp_val(ensemble, theta, u_depth, n_qubits, no_of_copies_per_eval = 0, n_B = 0):
  if n_B == 0:
    n_B = n_qubits
    
  final_estimate = 0
  ensemble = u_inverse(ensemble, theta, u_depth, n_qubits)
    
  for pair in ensemble:
    state = pair[1]
    exp_val = 0
    squarer = lambda t: np.abs(t) ** 2
    amplitudes = np.array([squarer(xi) for xi in state])
    for i in range(n_B):
      exp_temp = 0
      temp = (2 ** (n_qubits - i - 1))
      for j in range(0, 2 ** (i + 1), 2):
        exp_temp += sum(amplitudes[j * temp: (j + 1) * temp])

      final_estimate += pair[0] * np.real(exp_temp/n_B)
  if no_of_copies_per_eval != 0:
    return np.mean(np.random.choice(2, no_of_copies_per_eval, p = [1 - final_estimate, final_estimate]))
  else:
    return final_estimate
  
    
# This function carries out one iteration of SPSA algorithm for standard VQA
# 1. ensemble: A list of 2-tuples of the form [(p_1, \ket{\psi_1}), (p_2, \ket{\psi_2}), ... ,(p_t, \ket{\psi_t})]. Here, p_i are probabilities and \ket{\psi_i} are pure states in the mixture.
# 2. theta_tensor: An order 3 tensor of real parameters
# 3. u_depth: Depth of the ALA
# 4. n_qubits: Number of qubits involved
# 5. c_alpha: SPSA parameter
# 6. a_alpha: SPSA parameter
# 7. idx: k in SPSA
# 8. no_of_copies_per_eval: Number of measurement shots/number of copies per function evaluation. 0 means infinite shots/copies
# 9. n_B: Defined in u_inverse_exp_val
def single_spsa_update_vqa(ensemble, theta_tensor, u_depth, n_qubits, c_alpha, a_alpha, idx, no_of_copies_per_eval = 0, n_B=0):
  delta = np.random.choice([-1, 1], theta_tensor.shape[0] * theta_tensor.shape[1] * theta_tensor.shape[2], p = [1/2] * 2)
  delta = np.reshape(delta, (theta_tensor.shape[0], theta_tensor.shape[1], theta_tensor.shape[2]))

  c = 1/(idx ** c_alpha)
  a = 1/(idx ** a_alpha)

  plus = u_inverse_exp_val(ensemble, theta_tensor + (c * delta), u_depth, n_qubits, no_of_copies_per_eval = no_of_copies_per_eval, n_B=n_B)
  minus = u_inverse_exp_val(ensemble, theta_tensor - (c * delta), u_depth, n_qubits, no_of_copies_per_eval = no_of_copies_per_eval, n_B=n_B)

  return theta_tensor + (a * np.reciprocal((2 * c * delta)/(plus - minus)))



# This function computes T Classical Shadows of a mixed state and returns it as a list of lists.
# 1. ensemble: A list of 2-tuples of the form [(p_1, \ket{\psi_1}), (p_2, \ket{\psi_2}), ... ,(p_t, \ket{\psi_t})]. Here, p_i are probabilities and \ket{\psi_i} are pure states in the mixture.
# 2. T: Number of Classical shadows to be computed
# 3. verbose: Set 1 if you need print statements reporting the current iteration
def generate_shadows(ensemble, T, verbose = 0):
  num_qubits = int(np.log2(len(ensemble[0][1])))
  shadow_list = []
  gate_set_list = ['H', 'Y_basis']
  
  for j in range(T):
    if verbose == 1:
      if j % 1000 == 0:
        print("Currently generating shadow number: {}".format(j))
    temp_mixture = ensemble.copy()
    final_prob_array = np.zeros(2 ** num_qubits)
    gate_list = []
    for i in range(num_qubits):
      temp = np.random.choice(3, 1, p=[1/3, 1/3, 1/3])[0]
      if temp != 2:
        gate = gate_set_list[temp]
        gate_list.append((gate, i))
        for k in range(len(temp_mixture)):
          temp_state = across_qubits_gate(temp_mixture[k][1], [i], clifford_gate_dict[gate])
          temp_mixture[k] = (temp_mixture[k][0], temp_state)
      else:
        gate_list.append(('I', i))
    for pair in temp_mixture:
      prob_array = np.array([np.abs(pair[1][k]) ** 2 for k in range(len(pair[1]))])
      final_prob_array += pair[0] * prob_array
    outcome = np.random.choice(2 ** num_qubits, 1, p = list(final_prob_array))[0]
    outcome_bin = binary(outcome, num_qubits)
    final_outcome = []
    for i in range(len(gate_list)):
      if gate_list[i][0] == 'Y_basis':
        temp1 = clifford_gate_dict['Y_basis_dag']
      else:
        temp1 = clifford_gate_dict[gate_list[i][0]]
      temp = temp1[:, outcome_bin[i]]
      final_outcome.append(np.array(3 * np.outer(temp, np.conj(temp))) - Id)
    shadow_list.append(final_outcome)
  return shadow_list



# This function computes all reduced shadows of the form \hat{\rho} \big\_{A_i}, and returns a flattenned array with all of them concatenated
# 1. shadows: Output of generate_shadows
# 2. u_depth: Depth of the ALA
# 3. n_B: Defined in u_inverse_exp_val
def prepping_shadows(shadows, u_depth, n_B = 0):

  n_qubits = len(shadows[0])       
  if n_B == 0:
    n_B = n_qubits
  ans = np.zeros((1, ((4 ** u_depth) ** 2) * n_B), dtype = 'complex128')
  for single_shadow in shadows: 
    temp_list = []                                                                                                                                                                                                                                                                                                          
    for i in range(0, n_B, 2):
      state = np.kron(single_shadow[i], single_shadow[i + 1])
      for j in range(1, u_depth):
        state = np.kron(single_shadow[(i - j)%n_qubits], state)
        state = np.kron(state, single_shadow[(i + 1 + j)%n_qubits])

      temp_list.append(state.flatten())
      temp_list.append(state.flatten())

    ans += np.hstack(temp_list)
  return ans/len(shadows)

# This function returns the list of indices of the theta_tensor that does not get cancelled in the application of an ALA ansatz
# 1. u_depth: Depth of the ALA
# 2. n_qubits: Number of qubits involved
# 3. n_B: Defined in u_inverse_exp_val
def generate_obs_lotr(u_depth, n_qubits, n_B = 0):
  main_list = []
  if n_B == 0:
    n_B = n_qubits
  for observable_qubit_index in range(int(n_B/2)):
    list_of_thetas_remaining = []

    for d in range(u_depth):
      k = -1
      temp_list = []
      for i in range(d + 1):
        temp_list.append(((observable_qubit_index - (k * int((i + 1)/2))) % int(n_qubits/2), d))
        k *= -1
      list_of_thetas_remaining.append(temp_list)
    main_list.append(list_of_thetas_remaining)
  return main_list

# This function generates two observables of the form \ket{0}_i \bra{0}, where i = u_depth and u_depth + 1.
# 1. u_depth: Depth of the ALA
def generate_obs_list(u_depth):

  obs1 = np.kron(ket_0_density, Id)
  obs2 = np.kron(Id, ket_0_density)

  for i in range(1, u_depth):
    obs1 = np.kron(Id, obs1)
    obs1 = np.kron(obs1, Id)

    obs2 = np.kron(Id, obs2)
    obs2 = np.kron(obs2, Id)

  obs1 = np.array(obs1)
  obs2 = np.array(obs2)

  return obs1, obs2



# This function computes \hat{f}_{\rho, J}(theta_tensor) in ALSO
# 1. shadow_matrix_prepped: A row vector vcomprising of all the reduced density matrices \hat{\rho} \big|_{A_i}. Output of prepping_shadows
# 2. list_of_thetas: Output of generate_obs_lotr
# 3. theta_tensor: An order 3 tensor of real parameters
# 4. u_depth: Depth of the ALA
# 5. n_qubits: Number of qubits involved
# 6. obs: Output of generate_obs_lot
# 7. dtype: Data type involved in the computations
# 8. n_B: Defined in u_inverse_exp_val
def also_feval(shadow_matrix_prepped, list_of_thetas, theta_tensor, u_depth, n_qubits, obs, dtype = 'complex128', n_B=0):

  if n_B == 0:
    n_B = n_qubits
  obs_list = []
  Iden = np.eye(2, dtype=dtype)
  for observable_qubit_index in range(int(n_B/2)):
      obs1 = np.copy(obs[0])
      obs2 = np.copy(obs[1])
      q = 2
      for i in range(u_depth):
        sign = 1
        for j in range(len(list_of_thetas[observable_qubit_index][i])):
            if i%2==1:
              if j == len(list_of_thetas[observable_qubit_index][i])-2 and i != 0:
                obs1 = np.array(np.kron(obs1, Iden))
                obs2 = np.array(np.kron(obs2, Iden))
                q += 1
                s = s_gate(theta_tensor[list_of_thetas[observable_qubit_index][i][j][0], list_of_thetas[observable_qubit_index][i][j][1], :], dtype = dtype)
                obs1 = rotate_state(obs1, [q-2, q-1], s)
                obs2 = rotate_state(obs2, [q-2, q-1], s)

              elif j == len(list_of_thetas[observable_qubit_index][i])-1 and i!= 0:
                obs1 = np.kron(Iden, obs1)
                obs2 = np.kron(Iden, obs2)
                q += 1
                s = s_gate(theta_tensor[list_of_thetas[observable_qubit_index][i][j][0], list_of_thetas[observable_qubit_index][i][j][1], :], dtype = dtype)
                obs1 = rotate_state(obs1, [0, 1], s)
                obs2 = rotate_state(obs2, [0, 1], s)
              else:
                s = s_gate(theta_tensor[list_of_thetas[observable_qubit_index][i][j][0], list_of_thetas[observable_qubit_index][i][j][1], :], dtype = dtype)
                obs1 = rotate_state(obs1, [int((q-1)/2) + (sign * 2 * int((j+1)/2)) + (i%2), int(q/2) + (sign * 2 * int((j+1)/2)) + (i%2)], s)
                obs2 = rotate_state(obs2, [int((q-1)/2) + (sign * 2 * int((j+1)/2)) + (i%2), int(q/2) + (sign * 2 * int((j+1)/2)) + (i%2)], s)
                sign *= -1
    
            else:
              if j == len(list_of_thetas[observable_qubit_index][i])-1 and i != 0:
                obs1 = np.array(np.kron(obs1, Iden))
                obs2 = np.array(np.kron(obs2, Iden))
                q += 1
                s = s_gate(theta_tensor[list_of_thetas[observable_qubit_index][i][j][0], list_of_thetas[observable_qubit_index][i][j][1], :], dtype = dtype)                

                obs1 = rotate_state(obs1, [q-2, q-1], s)
                obs2 = rotate_state(obs2, [q-2, q-1], s)

              elif j == len(list_of_thetas[observable_qubit_index][i])-2 and i!= 0:
                obs1 = np.kron(Iden, obs1)
                obs2 = np.kron(Iden, obs2)
                q += 1
                s = s_gate(theta_tensor[list_of_thetas[observable_qubit_index][i][j][0], list_of_thetas[observable_qubit_index][i][j][1], :], dtype = dtype)
                obs1 = rotate_state(obs1, [0, 1], s)
                obs2 = rotate_state(obs2, [0, 1], s)
              else:
                s = s_gate(theta_tensor[list_of_thetas[observable_qubit_index][i][j][0], list_of_thetas[observable_qubit_index][i][j][1], :], dtype = dtype)
                obs1 = rotate_state(obs1, [int((q-1)/2) + (sign * 2 * int((j+1)/2)), int(q/2) + (sign * 2 * int((j+1)/2))], s)
                obs2 = rotate_state(obs2, [int((q-1)/2) + (sign * 2 * int((j+1)/2)), int(q/2) + (sign * 2 * int((j+1)/2))], s)
                sign *= -1

                
      obs_list.append(obs1.flatten())
      obs_list.append(obs2.flatten())
    
  obs_matrix = np.hstack(obs_list).T
  exp_list = np.real(np.matmul(np.conj(shadow_matrix_prepped), obs_matrix)/n_B)
  return exp_list[0]


# This function carries out one iteration of SPSA algorithm for ALSO
# 1. shadow_matrix_prepped: Output of prepping_shadows
# 2. list_of_thetas: Output of generate_obs_lotr
# 3. theta_tensor: An order 3 tensor of real parameters
# 4. u_depth: Depth of the ALA
# 5. n_qubits: Number of qubits involved
# 6. obs: Output of generate_obs_lot
# 7. c_alpha: SPSA parameter
# 8. a_alpha: SPSA parameter
# 9. idx: k in SPSA
# 10. dtype: Data type involved in the computations
# 11. n_B: Defined in u_inverse_exp_val
def single_spsa_update_also(shadow_matrix_prepped, list_of_thetas, theta_tensor, u_depth, n_qubits, obs, c_alpha, a_alpha, idx, dtype = 'complex128', n_B=0):
  delta = np.random.choice([-1, 1], theta_tensor.shape[0] * theta_tensor.shape[1] * theta_tensor.shape[2], p = [1/2] * 2)
  delta = np.reshape(delta, (theta_tensor.shape[0], theta_tensor.shape[1], theta_tensor.shape[2]))

  c = 1/(idx ** c_alpha)
  a = 1/(idx ** a_alpha)

  plus = also_feval(shadow_matrix_prepped, list_of_thetas, theta_tensor + (c * delta), u_depth, n_qubits, obs, dtype = dtype, n_B=n_B)
  minus = also_feval(shadow_matrix_prepped, list_of_thetas, theta_tensor - (c * delta), u_depth, n_qubits, obs, dtype = dtype, n_B=n_B)

  return theta_tensor + (a * np.reciprocal((2 * c * delta)/(plus - minus)))


# This function generates a data structure that allows for efficient simulation of large circuits (30 qubit) with computational basis states being input to ALAs.
# 1. bit_string: A bit string given as a list 
def bit_string_state_prepped(bit_string):
  temp_list = []                                                                                                                                                                                                                                                                                                          
  for i in range(0, n_qubits, 2):
    state = np.kron(Id[bit_string[i]], Id[bit_string[i+1]])
    for j in range(1, u_depth):
      state = np.kron(Id[bit_string[(i - j)%n_qubits]], state)

      state = np.kron(state, Id[bit_string[(i + 1 + j)%n_qubits]])
    temp_list.append(np.array(state)[0])

  return temp_list

# This function returns a list of qubits that the subcircuits of an ALA that does not cancel off acts on
# 1. u_depth: Depth of the ALA
# 2. n_qubits: Number of qubits involved
def generate_loqta(u_depth, n_qubits):
  list_of_qubits_to_act = []
  for j in range(u_depth):
    k = -1
    temp_list_index = []
    for i in range(j + 1):
      temp_list_index.append([(u_depth - 1 - (k * 2 * int((i + 1)/2)) + (j % 2)) % n_qubits, (u_depth - 1 - (k * 2 * int((i + 1)/2)) + 1 + (j % 2)) % n_qubits])
      k *= -1
    list_of_qubits_to_act.append(temp_list_index)
  return list_of_qubits_to_act

# This function applies the inverse of an ALA on an ensemble and returns f_{\rho,J}, J = \sum \limits_{i=1}^n_B \ket{0}_i\bra{0}. Suitable for large circuits.
# 1. prepped_bit_string_ensemble: Output of bit_string_state_prepped
# 2. list_of_qubits_to_act: outpuO of generate_loqta
# 3. list_of_thetas: outpuO of generate_obs_lotr
# 4. theta_tensor: An order 3 tensor of real parameters
# 5. u_depth: Depth of the ALA
# 6. n_qubits: Number of qubits involved
# 7. no_of_copies_per_eval: Number of measurement shots/number of copies per function evaluation. 0 means infinite shots/copies.
# 8. n_B: Defined in u_inverse_exp_val
def u_inverse_exp_val_efficient(prepped_bit_string_ensemble, list_of_qubits_to_act, list_of_thetas, theta_tensor, u_depth, n_qubits, no_of_copies_per_eval = 0, n_B = 0):
    if n_B == 0:
      n_B = n_qubits
    final_exp = 0
    for pair in prepped_bit_string_ensemble:

      for observable_qubit_index in range(int(n_B/2)):

        state = pair[1][observable_qubit_index]
        for i in range(u_depth-1,-1,-1):
          for j in range(len(list_of_thetas[observable_qubit_index][i])-1,-1,-1):
            s = s_gate_inverse(theta_tensor[list_of_thetas[observable_qubit_index][i][j][0], list_of_thetas[observable_qubit_index][i][j][1], :])
            state = across_qubits_gate(state, list_of_qubits_to_act[i][j], s)

        squarer = lambda t: np.abs(t) ** 2
        amplitudes = np.array([squarer(xi) for xi in state])

        for i in range(list_of_qubits_to_act[0][0][0], list_of_qubits_to_act[0][0][1] + 1):
          exp_temp = 0
          temp = (2 ** ((2 * u_depth) - i - 1))
          for j in range(0, 2 ** (i + 1), 2):
            exp_temp += sum(amplitudes[j * temp: (j + 1) * temp])

          final_exp += pair[0] * (exp_temp/n_B)
    if no_of_copies_per_eval != 0:
      return np.mean(np.random.choice(2, no_of_copies_per_eval, p = [1 - final_exp, final_exp]))

    else:
      return final_exp


# This function carries out one iteration of SPSA algorithm for standard VQA. Suitable when n_qubits is large and input is a bit string.
# 1. prepped_bit_string_ensemble: Output of bit_string_state_prepped
# 2. list_of_qubits_to_act: Output of generate_loqta
# 3. list_of_thetas: Output of generate_obs_lotr
# 4. theta_tensor: An order 3 tensor of real parameters
# 5. u_depth: Depth of the ALA
# 6. n_qubits: Number of qubits involved
# 7. c_alpha: SPSA parameter
# 8. a_alpha: SPSA parameter
# 9. idx: k in SPSA
# 10. no_of_copies_per_eval: Number of measurement shots/number of copies per function evaluation. 0 means infinite shots/copies.
# 11. n_B: Defined in u_inverse_exp_val
def single_spsa_update_vqa_efficient(prepped_bit_string_ensemble, list_of_qubits_to_act, list_of_thetas, theta_tensor, u_depth, n_qubits, c_alpha, a_alpha, idx, no_of_copies_per_eval = 0, n_B = 0):
  delta = np.random.choice([-1, 1], theta_tensor.shape[0] * theta_tensor.shape[1] * theta_tensor.shape[2], p = [1/2] * 2)
  delta = np.reshape(delta, (theta_tensor.shape[0], theta_tensor.shape[1], theta_tensor.shape[2]))

  c = 1/(idx ** c_alpha)
  a = 1/(idx ** a_alpha)

  plus = u_inverse_exp_val_efficient(prepped_bit_string_ensemble, list_of_qubits_to_act, list_of_thetas, theta_tensor + (c * delta), u_depth, n_qubits, no_of_copies_per_eval = no_of_copies_per_eval, n_B = n_B)
  minus = u_inverse_exp_val_efficient(prepped_bit_string_ensemble, list_of_qubits_to_act, list_of_thetas, theta_tensor - (c * delta), u_depth, n_qubits, no_of_copies_per_eval = no_of_copies_per_eval, n_B = n_B)

  return theta_tensor + (a * np.reciprocal((2 * c * delta)/(plus - minus)))


# This function generates a set of one qubit operators from which shadows of bit string state can be generated very fast
# 1. dtype: Data type of the entries of the matrices involved
def prepare_sampling_set(dtype = 'complex128'):
    sampling_set = []
    arr = (3 * (H_gate[:,0] @ np.conj(H_gate[:,0]).T)) - np.eye(2)
    sampling_set.append(arr.astype(dtype))
    arr = (3 * (H_gate[:,1] @ np.conj(H_gate[:,1]).T)) - np.eye(2)
    sampling_set.append(arr.astype(dtype))
    arr = (3 * (Y_basis_dag[:,0] @ np.conj(Y_basis_dag[:,0]).T)) - np.eye(2)
    sampling_set.append(arr.astype(dtype))
    arr = (3 * (Y_basis_dag[:,1] @ np.conj(Y_basis_dag[:,1]).T)) - np.eye(2)
    sampling_set.append(arr.astype(dtype))
    arr = (3 * (Id[:,0] @ np.conj(Id[:,0]).T)) - np.eye(2)
    sampling_set.append(arr.astype(dtype))
    arr = (3 * (Id[:,1] @ np.conj(Id[:,1]).T)) - np.eye(2)
    sampling_set.append(arr.astype(dtype))

    return sampling_set

# This function computes T Classical Shadows of a computational basis state given as a bit string and returns it as a list of lists.
# 1. bit_string: A bit string given as a list 
# 2. T: Number_of_shadows 
# 3. dtype: Data type of the entries of the matrices involved
def bit_string_shadows(bit_string, T, dtype = 'complex128'):
  list_of_shadows = []
    
  n_qubits = len(bit_string)
  sampling_set = prepare_sampling_set(dtype=dtype)

  for i in range(n_qubits):
    print("Current Qubit: {}".format(i))
    if bit_string[i] == 0:
      list_of_shadows.append(random.choices(sampling_set, weights = [1/6, 1/6, 1/6, 1/6, 1/3, 0], k = T))

    else:
      list_of_shadows.append(random.choices(sampling_set, weights = [1/6, 1/6, 1/6, 1/6, 0, 1/3], k = T))

  print("Transpoing Lists")
  list_of_shadows = list(map(list, zip(*list_of_shadows)))
  return list_of_shadows

# This function computes the reduced shadows (similar to shadows_prepped) but specifically for QAE for large number of qubits.
# 1. shadows_list: Output of bit_string_shadows
# 2. u_depth: Depth of the ALA
# 3. n_B: Defined in u_inverse_exp_val
def prepping_shadows_30q_qae(shadows_list, u_depth, n_B = 0):
  n_qubits = len(shadows_list[0][0])       
  if n_B == 0:
    n_B = n_qubits
  ans = np.zeros((1, ((4 ** u_depth) ** 2) * n_B), dtype = 'complex128')
  for k in range(len(shadows_list[0])):
    s = np.random.choice(2,p=[1/3,2/3])
    if k%1000 == 0:
      print("Current shadow: {}".format(k))
    temp_list = []                                                                                                                                                                                                                                                                                                          
    for i in range(0, n_B, 2):
      state = np.kron(shadows_list[s][k][i], shadows_list[s][k][i + 1])
      for j in range(1, u_depth):
        state = np.kron(shadows_list[s][k][(i - j)%n_qubits], state)
        state = np.kron(state, shadows_list[s][k][(i + 1 + j)%n_qubits])

      temp_list.append(state.flatten())
      temp_list.append(state.flatten())

    ans += np.hstack(temp_list)
  return ans/len(shadows_list[0])






## Qiskit related

Here, we define the functions required for the Matrix Product State simulations

In [5]:
# This function applies the inverse of s sub circuit to a qiskit circuit
# 1. qc: Qiskit quantum circuit object
# 2. q1: Qubit 1
# 3. q2: Qubit 2
# 4. theta: A 12 dimensional numpy array
def s_ansatz_inverse_qiskit(qc, q1, q2, theta):
  qc.cnot(q2, q1)
  qc.rx(-theta[11], q2)
  qc.ry(-theta[10], q2)
  qc.rx(-theta[9], q2)

  qc.rx(-theta[8], q1)
  qc.ry(-theta[7], q1)
  qc.rx(-theta[6], q1)

  qc.cnot(q1, q2)

  qc.rx(-theta[5], q2)
  qc.ry(-theta[4], q2)
  qc.rx(-theta[3], q2)

  qc.rx(-theta[2], q1)
  qc.ry(-theta[1], q1)
  qc.rx(-theta[0], q1)
  return qc

# This function applies the s sub circuit to a qiskit circuit
# 1. qc: Qiskit quantum circuit object
# 2. q1: Qubit 1
# 3. q2: Qubit 2
# 4. theta: A 12 dimensional numpy array
def s_ansatz_qiskit(qc, q1, q2, theta):
  qc.rx(theta[0], q1)
  qc.ry(theta[1], q1)
  qc.rx(theta[2], q1)
  
  qc.rx(theta[3], q2)
  qc.ry(theta[4], q2)
  qc.rx(theta[5], q2)

  qc.cnot(q1, q2)

  qc.rx(theta[6], q1)
  qc.ry(theta[7], q1)
  qc.rx(theta[8], q1)
  
  qc.rx(theta[9], q2)
  qc.ry(theta[10], q2)
  qc.rx(theta[11], q2)

  qc.cnot(q2, q1)
  return qc

# This function applies an ALA ansatz to a qiskit circuit
# 1. qc: Qiskit quantum circuit object
# 2. theta_tensor: An order 3 tensor of real parameters
# 3. u_depth: Depth of the ALA
# 4. n_qubits: Number of qubits involved
def u_inverse_qiskit(qc, theta_tensor, u_depth, n_qubits):
  for d in range(u_depth-1, -1, -1):
    for i in range(n_qubits-2, -1, -2):
      qc = s_ansatz_inverse_qiskit(qc, i + (d%2), (i+1 + (d%2))%n_qubits, theta_tensor[int(i/2)][d])
  return qc


# This function computes the fidelity between a computational basis state and output of the inverse of an ALA ansatz
# 1. bit_string: A bit string given as a list 
# 2. theta_tensor: An order 3 tensor of real parameters
def fidelity_bs(bit_string, theta_tensor):
    u_depth = 4
    n_qubits = len(bit_string)
    simulator = AerSimulator(method='matrix_product_state')

    qr = QuantumRegister(n_qubits)
    qc = QuantumCircuit(qr)
    for i in range(n_qubits):
        if bit_string[i] == 1:
            qc.x(i)

    qc = u_inverse_qiskit(qc, theta_tensor, u_depth, n_qubits)
    qc.save_matrix_product_state(label='my_mps')

    tcirc = transpile(qc, simulator)
    result = simulator.run(tcirc).result()
    data = result.data(0)
    
    mat = np.matrix([[1]])
    for i in range(n_qubits):
        if i != n_qubits - 1:
            mat =  mat @ data['my_mps'][0][i][0]
            mat = np.multiply(mat, data['my_mps'][1][i])
        else:
            mat = mat @ data['my_mps'][0][i][0]

    return np.abs(mat[0,0])

# 8 Qubit

In this section, we provide code to implement ALSO and VQA on $8$ qubit SPP and QA problems.

## SPP

In [ ]:

n_qubits =  8 # Number of qubits the state should be prepared on 

u_depth = 3 # Depth of U(\theta)

answer_tensor = np.random.uniform(1, 2 * np.pi, (int(n_qubits/2), u_depth, 12)) # U(answer_tensor) \ket{0} is the state whose preparation circuit we want to find.
theta_tensor = np.random.uniform(-2 * np.pi, 2 * np.pi, (int(n_qubits/2), u_depth, 12)) # Initialize random values as parameters




In [ ]:

no_of_iter = 3000 # Total number of iterations in SPSA

c_alpha = 0.5 # SPSA parameter
a_alpha = 0.5 # SPSA parameter

In [ ]:
# Store the state U(answer_tensor) \ket{0}
state = np.array([1] + [0] * ((2 ** n_qubits) - 1))
ensemble = [(1, state)]
ensemble = u(ensemble, answer_tensor, u_depth, n_qubits) 


### ALSO

In [ ]:
shadows = generate_shadows(ensemble, 500000, verbose = 1) # Generate the shadows
shadows_prepped = prepping_shadows(shadows, u_depth) # Compute the reduced shadow states

In [ ]:
# This function implements the SPSA algorithm to solve SPP using ALSO
# 1. shadow_prepped: Output of prepping_shadows
# 2. theta_tensor: An order 3 tensor of real parameters
# 3. ensemble: A list of 2-tuples of the form [(p_1, \ket{\psi_1}), (p_2, \ket{\psi_2}), ... ,(p_t, \ket{\psi_t})]. Here, p_i are probabilities and \ket{\psi_i} are pure states in the mixture.
# 4. u_depth: depth of the ALA
# 5. n_qubits: Number of qubits involved
# 6. c_alpha: SPSA parameter
# 7. a_alpha: SPSA parameter
# 8. no_of_iter: Total number of iterations
# 9. dtype: Data type involved in the computations
# 10. interval: At what intervals should the infidelities be computed
def also_8q_spp(shadows_prepped, theta_tensor, ensemble, u_depth, n_qubits, c_alpha, a_alpha, no_of_iter, dtype='complex128', interval = 100):
    iter_no = 1
    infid_list = []
    theta_list = []
    obs1 = np.array(np.kron(np.matrix([[1,0],[0,0]]), np.eye(2))).astype(dtype)
    obs2 = np.array(np.kron(np.eye(2), np.matrix([[1,0],[0,0]]))).astype(dtype)
    obs = [obs1, obs2]
    list_of_thetas = generate_obs_lotr(u_depth, n_qubits)

    while iter_no < no_of_iter + 1:

        theta_tensor = single_spsa_update_also(shadows_prepped, list_of_thetas, theta_tensor, u_depth, n_qubits, obs, c_alpha, a_alpha, iter_no, dtype = dtype)
        if iter_no % interval == 0:
            print("Current iteration: {}".format(iter_no))
            init_state = np.array([1] + [0] * ((2 ** n_qubits) - 1))
            init_state = u([(1,init_state)], theta_tensor, u_depth, n_qubits)[0][1]
            infidelity = 1-np.abs(np.dot(np.conj(init_state), ensemble[0][1]))
            print("Current infidelity: {}".format(infidelity))
            infid_list.append(infidelity)
            theta_list.append(theta_tensor)

        iter_no += 1
    return infid_list, theta_list

In [ ]:
infid_list, theta_list = also_8q_spp(shadows_prepped, theta_tensor, ensemble, u_depth, n_qubits, c_alpha, a_alpha, no_of_iter)

In [ ]:
# Plot the results

plt.figure(figsize=(12,6))
plt.rcParams.update({'font.size': 20})

plt.plot(list(range(1, len(infid_list) + 1)), infid_list)

plt.yticks(np.arange(0,1.1,0.1))
plt.legend(loc="lower right")
plt.xlabel("Iteration number")
plt.ylabel("Infidelity")
plt.show()

### VQA

In [ ]:
# This function implements the SPSA algorithm to solve SPP using standard VQA
# 1. ensemble: A list of 2-tuples of the form [(p_1, \ket{\psi_1}), (p_2, \ket{\psi_2}), ... ,(p_t, \ket{\psi_t})]. Here, p_i are probabilities and \ket{\psi_i} are pure states in the mixture.
# 2. theta_tensor: An order 3 tensor of real parameters
# 3. u_depth: Depth of the ALA
# 4. n_qubits: Number of qubits involved
# 5. c_alpha: SPSA parameter
# 6. a_alpha: SPSA parameter
# 7. no_of_iter: Total number of iterations
# 8. no_of_copies_per_eval: Number of measurement shots/number of copies per function evaluation. 0 means infinite shots/copies.
# 9. interval: At what intervals should the infidelities be computed
def vqa_8q_spp(ensemble, theta_tensor, u_depth, n_qubits, c_alpha, a_alpha, no_of_iter, no_of_copies_per_eval = 0, interval=100):
    iter_no = 1
    infid_list = []
    theta_list = []
    while iter_no < no_of_iter + 1:

        theta_tensor = single_spsa_update_vqa(ensemble, theta_tensor, u_depth, n_qubits, c_alpha, a_alpha, iter_no, no_of_copies_per_eval = no_of_copies_per_eval)
        if iter_no % interval == 0:
            print("Current iteration: {}".format(iter_no))
            init_state = np.array([1] + [0] * ((2 ** n_qubits) - 1))
            init_state = u([(1,init_state)], theta_tensor, u_depth, n_qubits)[0][1]
            infidelity = 1-np.abs(np.dot(np.conj(init_state), ensemble[0][1]))
            print("Current infidelity: {}".format(infidelity))
            infid_list.append(infidelity)
            theta_list.append(theta_tensor)
        iter_no += 1
    return infid_list, theta_list

In [ ]:
infid_list, theta_list = vqa_8q_spp(ensemble, theta_tensor, u_depth, n_qubits, c_alpha, a_alpha, no_of_iter, no_of_copies_per_eval = 0)

In [ ]:
# Plot the results

plt.figure(figsize=(12,6))
plt.rcParams.update({'font.size': 20})

plt.plot(list(range(1, len(infid_list) + 1)), infid_list)

plt.yticks(np.arange(0,0.6,0.1))
plt.legend(loc="lower right")
plt.xlabel("Iteration number")
plt.ylabel("Infidelity")
plt.show()

## QA

In [ ]:

n_qubits =  8 # Number of qubits the state should be prepared on 

u_depth = 3 # Depth of U(\theta)

theta_tensor = np.random.uniform(-2 * np.pi, 2 * np.pi, (int(n_qubits/2), u_depth, 12)) # Initialize random values as parameters


In [ ]:
ensemble = []
p = np.array(list(range(1,3))) # Probability values 1/3 and 2/3
p = p/sum(p)
for i in range(2): # Generate two random ALA compatible states
    answer_tensor = np.random.uniform(1, 2 * np.pi, (int(n_qubits/2), u_depth, 12))
    state = np.array([1] + [0] * ((2 ** n_qubits) - 1))
    state = u([(1,state)], answer_tensor, u_depth, n_qubits)[0][1]
    ensemble.append((p[i], state))

In [ ]:

no_of_iter = 6000

c_alpha = 0.3
a_alpha = 0.3

### ALSO

In [ ]:
shadows = generate_shadows(ensemble, 500000, verbose = 1)
shadows_prepped = prepping_shadows(shadows, u_depth, n_B = 4)


In [ ]:
# SImilar to the spp case. Only difference is n_B = 4.
def also_8q_qa(ensemble, shadows_prepped, theta_tensor, u_depth, n_qubits, c_alpha, a_alpha, no_of_iter, n_B = 4, dtype='complex128', interval = 100):
    iter_no = 1
    cost_list = []
    theta_list = []
    list_of_qubits_to_act = generate_loqta(u_depth, n_qubits)
    list_of_thetas = generate_obs_lotr(u_depth, n_qubits)
    obs1 = np.array(np.kron(np.matrix([[1,0],[0,0]]), np.eye(2))).astype(dtype)
    obs2 = np.array(np.kron(np.eye(2), np.matrix([[1,0],[0,0]]))).astype(dtype)
    obs = [obs1, obs2]
    while iter_no < no_of_iter + 1:

        theta_tensor = single_spsa_update_also(shadows_prepped, list_of_thetas, theta_tensor, u_depth, n_qubits, obs, c_alpha, a_alpha, iter_no, n_B = n_B)
        if iter_no % interval == 0:
            print("Current iteration: {}".format(iter_no))
            cost = 1 - u_inverse_exp_val(ensemble, theta_tensor, u_depth, n_qubits, no_of_copies_per_eval = 0, n_B = 4)
            print("Current cost: {}".format(cost))
            cost_list.append(cost)
            theta_list.append(theta_tensor)
        iter_no += 1
    return cost_list, theta_list

In [ ]:
cost_list, theta_list = also_8q_qa(ensemble, shadows_prepped, theta_tensor, u_depth, n_qubits, c_alpha, a_alpha, no_of_iter, n_B = 4)

In [ ]:
# Plot the results

plt.figure(figsize=(12,6))
plt.rcParams.update({'font.size': 20})

plt.plot(list(range(1, len(cost_list) + 1)), cost_list)

plt.yticks(np.arange(0,1.1,0.1))
plt.legend(loc="lower right")
plt.xlabel("Iteration number")
plt.ylabel("Cost")
plt.show()

### VQA

In [ ]:
# SImilar to the spp case. Only difference is n_B = 4.
def vqa_8q_qa(ensemble, theta_tensor, u_depth, n_qubits, c_alpha, a_alpha, no_of_iter, no_of_copies_per_eval = 0, n_B = 0, interval = 100, dtype='complex128'):
    iter_no = 1
    cost_list = []
    theta_list = []
    while iter_no < no_of_iter + 1:
        theta_tensor = single_spsa_update_vqa(ensemble, theta_tensor, u_depth, n_qubits, c_alpha, a_alpha, iter_no, no_of_copies_per_eval = no_of_copies_per_eval, n_B=4)
        if iter_no % interval == 0:
            print("Current iteration: {}".format(iter_no))
            cost = 1 - u_inverse_exp_val(ensemble, theta_tensor, u_depth, n_qubits, no_of_copies_per_eval = 0, n_B = 4)
            print("Current cost: {}".format(cost))
            cost_list.append(cost)
            theta_list.append(theta_tensor)
        iter_no += 1
    return cost_list, theta_list

In [ ]:
cost_list, theta_list = vqa_8q_qa(ensemble, theta_tensor, u_depth, n_qubits, c_alpha, a_alpha, no_of_iter, no_of_copies_per_eval = 0, n_B = 4)

In [ ]:
# Plot the results

plt.figure(figsize=(12,6))
plt.rcParams.update({'font.size': 20})

plt.plot(list(range(1, len(cost_list) + 1)), cost_list)

plt.yticks(np.arange(0,0.6,0.1))
plt.legend(loc="lower right")
plt.xlabel("Iteration number")
plt.ylabel("Cost")
plt.show()

# 30 qubit

In this section, we provide code to implement ALSO and VQA on $30$ qubit SPP and QA problems.

## SPP

In [ ]:

n_qubits = 30# Number of qubits the state should be prepared on 

u_depth = 4

theta_tensor = np.random.uniform(-2 * np.pi, 2 * np.pi, (int(n_qubits/2), u_depth, 12)) # Initialize random values as parameter


In [ ]:

no_of_iter = 9000

c_alpha = 0.5
a_alpha = 0.5

In [ ]:
# Define the computational state as a bit string
bit_string = []
for i in range(n_qubits):
    if i%3 == 0:
        bit_string.append(1)
    else:
        bit_string.append(0)


### ALSO

In [ ]:
shadows = bit_string_shadows(bit_string, 500000)

In [ ]:
shadows_prepped = prepping_shadows(shadows, u_depth) # This process can take a few hours when T=500000. But it is a one-time cost. Once the reduced shadow states are loaded, we can carry out optimization in many different ways on the shadows_prepped itself.

In [ ]:
# Similar function to the 8 qubit case. Only difference is instead of ensemble, we pass the bit string here.
def also_30q_spp(shadows_prepped, theta_tensor, bit_string, u_depth, n_qubits, c_alpha, a_alpha, no_of_iter ,interval=10, dtype='complex128'):
    iter_no = 1

    infid_list = []
    theta_list = []

    list_of_thetas = generate_obs_lotr(u_depth, n_qubits)
    obs1 = np.array(np.kron(np.matrix([[1,0],[0,0]]), np.eye(2))).astype(dtype)
    obs2 = np.array(np.kron(np.eye(2), np.matrix([[1,0],[0,0]]))).astype(dtype)
    obs = [obs1, obs2]
    
    while iter_no < no_of_iter + 1:    
      theta_tensor = single_spsa_update_also(shadows_prepped, list_of_thetas, theta_tensor, u_depth, n_qubits, obs, c_alpha, a_alpha, iter_no, dtype = dtype)      
      if iter_no % interval == 0:
        print("Current iteration: {}".format(iter_no))
        infidelity = 1-fidelity_bs(bit_string, theta_tensor)
        print("Current infidelity: {}".format(infidelity))
        theta_list.append(theta_tensor.flatten())
      iter_no += 1
    return infid_list, theta_list

In [ ]:
infid_list, theta_list = also_30q_spp(shadows_prepped, theta_tensor, bit_string, u_depth, n_qubits, c_alpha, a_alpha, no_of_iter)

In [ ]:
# Plot the results

plt.figure(figsize=(12,6))
plt.rcParams.update({'font.size': 20})

plt.plot(list(range(1, len(infid_list) + 1)), infid_list)

plt.yticks(np.arange(0,1.1,0.1))
plt.legend(loc="lower right")
plt.xlabel("Iteration number")
plt.ylabel("Infidelity")
plt.show()

### VQA

In [ ]:
# Similar function to the 8 qubit case. Only difference is instead of ensemble, we pass the bit string here.
def vqa_30q_spp(bit_string, theta_tensor, u_depth, c_alpha, a_alpha, no_of_iter, no_of_copies_per_eval = 0, interval=1000):
    list_of_qubits_to_act = generate_loqta(u_depth, n_qubits)
    list_of_thetas = generate_obs_lotr(u_depth, n_qubits)
    prepped_bit_string_ensemble = [(1,bit_string_state_prepped(bit_string))]

    iter_no = 1
    infid_list = []
    theta_list = []
    while iter_no < no_of_iter + 1:

      
      theta_tensor = single_spsa_update_vqa_efficient(prepped_bit_string_ensemble, list_of_qubits_to_act, list_of_thetas, theta_tensor, u_depth, n_qubits, c_alpha, a_alpha, iter_no, no_of_copies_per_eval = no_of_copies_per_eval)
      if iter_no % interval == 0:
        print("Current iteration: {}".format(iter_no))        
        infidelity = 1-fidelity_bs(bit_string, theta_tensor)
        print("Current infidelity: {}".format(infidelity))
        infid_list.append(infidelity)
        theta_list.append(theta_tensor.flatten())
      iter_no += 1


    return infid_list, theta_list

In [ ]:
infid_list, theta_list = vqa_30q_spp(bit_string, theta_tensor, u_depth, c_alpha, a_alpha, no_of_iter, no_of_copies_per_eval = 10000)

In [ ]:
# Plot the results

plt.figure(figsize=(12,6))
plt.rcParams.update({'font.size': 20})

plt.plot(list(range(1, len(infid_list) + 1)), infid_list)

plt.yticks(np.arange(0,1.1,0.1))
plt.legend(loc="lower right")
plt.xlabel("Iteration number")
plt.ylabel("Infidelity")
plt.show()

## QA

In [ ]:

n_qubits = 30# Number of qubits the state should be prepared on 

u_depth = 4

theta_tensor = np.random.uniform(-2 * np.pi, 2 * np.pi, (int(n_qubits/2), u_depth, 12)) # Initialize random values as parameters




In [ ]:

no_of_iter = 9000

c_alpha = 0.5
a_alpha = 0.5

In [ ]:
# Define two computational state as a bit string
bit_string1 = []
for i in range(n_qubits):
    if i%3 == 0:
        bit_string1.append(1)
    else:
        bit_string1.append(0)

bit_string2 = []
for i in range(n_qubits):
    if (i+1)%3 == 0:
        bit_string2.append(1)
    else:
        bit_string2.append(0)


prepped_bit_string_ensemble = [(1/3,bit_string_state_prepped(bit_string1)), (2/3,bit_string_state_prepped(bit_string2))] # Creates an ensemble of data structres capable of simulating large ALA circuits acting on computational basis states efficiently

### ALSO

In [ ]:
shadows1 = bit_string_shadows(bit_string1, 500000)
shadows2 = bit_string_shadows(bit_string2, 500000)

In [ ]:
shadows_prepped = prepping_shadows_30q_qae([shadows1, shadows2], u_depth, n_B = 10) # Note that reduced shadow states are computed using here. # This process can take a few hours when T=500000. But it is a one-time cost. Once the reduced shadow states are loaded, we can carry out optimization in many different ways on the shadows_prepped itself.

In [ ]:
# Similar function to the 8 qubit case. Only difference is instead of ensemble, we pass prepped_bit_string_ensemble here.

def also_30q_qa(prepped_bit_string_ensemble, shadows_prepped, theta_tensor, u_depth, n_qubits, c_alpha, a_alpha, no_of_iter, n_B = 10, interval = 100, dtype='complex128'):
    list_of_qubits_to_act = generate_loqta(u_depth, n_qubits)
    list_of_thetas = generate_obs_lotr(u_depth, n_qubits)
    obs1 = np.array(np.kron(np.matrix([[1,0],[0,0]]), np.eye(2))).astype(dtype)
    obs2 = np.array(np.kron(np.eye(2), np.matrix([[1,0],[0,0]]))).astype(dtype)
    obs = [obs1, obs2]
    iter_no = 1
    cost_list = []
    theta_list = []
    
    while iter_no < no_of_iter + 1:  
      theta_tensor = single_spsa_update_also(shadows_prepped, list_of_thetas, theta_tensor, u_depth, n_qubits, obs, c_alpha, a_alpha, iter_no, dtype = dtype, n_B=10)
      if iter_no % interval == 0:
          print("Current iteration: {}".format(iter_no))
          cost = 1-u_inverse_exp_val_efficient(prepped_bit_string_ensemble, list_of_qubits_to_act, list_of_thetas, theta_tensor, u_depth, n_qubits, no_of_copies_per_eval = 0, n_B = 10)
          print("Current cost: {}".format(cost))
          cost_list.append(cost)
          theta_list.append(theta_tensor.flatten())
      iter_no += 1
    return cost_list, theta_list

In [ ]:
cost_list, theta_list = also_30q_qa(prepped_bit_string_ensemble, shadows_prepped, theta_tensor, u_depth, n_qubits, c_alpha, a_alpha, no_of_iter, n_B = 10)

In [ ]:
# Plot the results

plt.figure(figsize=(12,6))
plt.rcParams.update({'font.size': 20})

plt.plot(list(range(1, len(cost_list) + 1)), cost_list)

plt.yticks(np.arange(0,0.6,0.1))
plt.legend(loc="lower right")
plt.xlabel("Iteration number")
plt.ylabel("Cost")
plt.show()

### VQA

In [ ]:
# Similar function to the 8 qubit case. Only difference is instead of ensemble, we pass prepped_bit_string_ensemble here.

def vqa_30q_qa(prepped_bit_string_ensemble, theta_tensor, u_depth, n_qubits, c_alpha, a_alpha, no_of_iter, no_of_copies_per_eval = 0, n_B = 10, dtype='complex128',interval=100):
    list_of_qubits_to_act = generate_loqta(u_depth, n_qubits)
    list_of_thetas = generate_obs_lotr(u_depth, n_qubits)
    iter_no = 1
    cost_list = []
    theta_list = []

    while iter_no < no_of_iter + 1:
      theta_tensor = single_spsa_update_vqa_efficient(prepped_bit_string_ensemble, list_of_qubits_to_act, list_of_thetas, theta_tensor, u_depth, n_qubits, c_alpha, a_alpha, iter_no, no_of_copies_per_eval = no_of_copies_per_eval, n_B = n_B)
      if iter_no % interval == 0:
        print("Current iteration: {}".format(iter_no))
        cost = 1-u_inverse_exp_val_efficient(prepped_bit_string_ensemble, list_of_qubits_to_act, list_of_thetas, theta_tensor, u_depth, n_qubits, no_of_copies_per_eval = 0, n_B = n_B)
        print("Current cost: {}".format(cost))
        cost_list.append(cost)
        theta_list.append(theta_tensor.flatten())
      iter_no += 1

    return cost_list, theta_list

In [ ]:
cost_list, theta_list = vqa_30q_qa(prepped_bit_string_ensemble, theta_tensor, u_depth, n_qubits, c_alpha, a_alpha, no_of_iter, no_of_copies_per_eval = 0, n_B = 10, dtype='complex128',interval=100)

In [ ]:
# Plot the results

plt.figure(figsize=(12,6))
plt.rcParams.update({'font.size': 20})

plt.plot(list(range(1, len(cost_list) + 1)), cost_list)

plt.yticks(np.arange(0,0.6,0.1))
plt.legend(loc="lower right")
plt.xlabel("Iteration number")
plt.ylabel("Cost")
plt.show()

# Powell's method

## SPP

In [ ]:
n_qubits =  8 # Number of qubits the state should be prepared on 

s_depth = 1
u_depth = 3

answer_tensor = np.random.uniform(1, 2 * np.pi, (int(n_qubits/2), u_depth, 12 * s_depth))
theta_tensor = np.random.uniform(-2 * np.pi, 2 * np.pi, (int(n_qubits/2), u_depth, 12 * s_depth)) # Initialize random values as parameters

state = np.array([1] + [0] * ((2 ** n_qubits) - 1))
ensemble = [(1, state)]
ensemble = u(ensemble, answer_tensor, u_depth, n_qubits)

### ALSO

In [ ]:
list_of_thetas = generate_obs_lotr(u_depth, n_qubits)
obs1 = np.array(np.kron(np.matrix([[1,0],[0,0]]), np.eye(2))).astype(dtype)
obs2 = np.array(np.kron(np.eye(2), np.matrix([[1,0],[0,0]]))).astype(dtype)
obs = [obs1, obs2]

In [ ]:
shadows = generate_shadows(ensemble, 500000, verbose = 1)
shadows_prepped = prepping_shadows(shadows, u_depth)

In [ ]:
# Our objective function is wrapped in this function so that it can be passed to scipy.minimize
def passable_function(theta):
  theta_reshaped = np.reshape(theta, (int(n_qubits/2), u_depth, 12))
  val = also_feval(shadows_prepped, list_of_thetas, theta_tensor, u_depth, n_qubits, obs)
  print("Current value: {}".format(val))
  return -val

In [ ]:

ans = minimize(fun = passable_function, x0 = theta_tensor, method='Powell')

### VQA

In [ ]:
# Our objective function is wrapped in this function so that it can be passed to scipy.minimize
no_of_copies_per_eval = 100000 # Measurement shots/number of copies per function evaluation
def passable_function(theta):
  theta_reshaped = np.reshape(theta, (int(n_qubits/2), u_depth, 12 * s_depth))
  val = u_inverse_exp_val(ensemble, theta, u_depth, n_qubits, no_of_copies_per_eval = no_of_copies_per_eval)
  print("Current value: {}".format(val))
  return -val

In [ ]:

ans = minimize(fun = passable_function, x0 = theta_tensor, method='Powell', options = {'maxfev':50000})

## QA

In [ ]:

n_qubits =  8# Number of qubits the state should be prepared on 

s_depth = 1
u_depth = 3

theta_tensor = np.random.uniform(-2 * np.pi, 2 * np.pi, (int(n_qubits/2), u_depth, 12)) # Initialize random values as parameters

ensemble = []
p = np.array(list(range(1,3)))
p = p/sum(p)
for i in range(2):
    state = np.array([1] + [0] * ((2 ** n_qubits) - 1))
    state = u([(1,state)], answer_tensor, u_depth, n_qubits)[0][1]
    ensemble.append((p[i], state))
    


### ALSO

In [ ]:
shadows = generate_shadows(ensemble, 500000, verbose = 1)
shadows_prepped = prepping_shadows(shadows, u_depth, n_B = 4)


In [ ]:
# Our objective function is wrapped in this function so that it can be passed to scipy.minimize
def passable_function(theta):
  theta_reshaped = np.reshape(theta, (int(n_qubits/2), u_depth, 12))
  val = also_feval(shadows_prepped, list_of_thetas, theta_tensor, u_depth, n_qubits, obs, n_B=4)
  print("Current value: {}".format(val))
  return -val

In [ ]:

ans = minimize(fun = passable_function, x0 = theta_tensor, method='Powell')

### VQA

In [ ]:
# Our objective function is wrapped in this function so that it can be passed to scipy.minimize

no_of_copies_per_eval = 100000 # Measurement shots/number of copies per function evaluation
def passable_function(theta):
  theta_reshaped = np.reshape(theta, (int(n_qubits/2), u_depth, 12 * s_depth))
  val = u_inverse_exp_val(ensemble, theta, u_depth, n_qubits, no_of_copies_per_eval = no_of_copies_per_eval, n_B = 4)
  print("Current value: {}".format(val))
  return -val

In [ ]:

ans = minimize(fun = passable_function, x0 = theta_tensor, method='Powell', options = {'maxfev':50000})